# ArcGIS StoryMap Classic-to-New Conversion: Debuggable Notebook Structure

This notebook is structured for efficient development and debugging in VS Code, while supporting easy adaptation to ArcGIS Online Notebooks with ipywidgets-based user input.  
**Sections:**  
1. Import Required Libraries and Setup  
2. Define Helper Functions for StoryMap Conversion  
3. Implement Core Conversion Logic as Functions  
4. VS Code Debug/Test Entry Point  
5. ipywidgets-based UI for ArcGIS Online Notebooks  
6. Conditional Execution: VS Code vs ArcGIS Online Notebook  

## 1. Import Required Libraries and Setup

Import all necessary libraries and set up configuration variables.  
Includes package installation logic for VS Code environments.

In [ ]:
# Check for required packages and install if necessary (VS Code only)
import sys

def ensure_package(package, import_name=None):
    import_name = import_name or package
    try:
        __import__(import_name)
    except ImportError:
        try:
            print(f"Installing {package} ...")
            !{sys.executable} -m pip install {package}
        except Exception as e:
            print(f"Could not install {package}: {e}")

required_packages = [
    ("webcolors", None),
    ("bs4", None),
    ("arcgis", None),
    ("Pillow", "PIL"),
    ("pandas", None),
    ("requests", None),
    ("ipywidgets", None)
]
for pkg, imp in required_packages:
    ensure_package(pkg, imp)

In [ ]:
# Import libraries and set up configuration variables
from bs4 import BeautifulSoup
from arcgis.apps.storymap import StoryMap, Themes, Image, Video, Audio, Embed, Map, Button, Text, Gallery, Timeline, Sidecar, Code, Table, TextStyles, Collection, CollectionNavigation
from arcgis.gis import GIS, Item
import pandas as pd
import webcolors
import requests
import os
import sys
from PIL import Image as PILImage
from io import BytesIO
import threading
import time
import re
import json

try:
    import ipywidgets as widgets
    from IPython.display import display
except ImportError:
    widgets = None
    display = print

# Set Pandas display options
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', 1000)

# Default thumbnail path
default_thumbnail_path = "https://cdn-a.arcgis.com/cdn/1BE082D/js/arcgis-app-components/arcgis-app/assets/arcgis-item-thumbnail/storymap.png"

# Environment flag: Set to True if running in ArcGIS Online Notebook
agoNotebook = False  # Change to True in Section 6 or via UI

## 2. Define Helper Functions for StoryMap Conversion

Reusable functions for HTML parsing, color conversion, thumbnail creation, etc.

In [ ]:
def color_to_hex(color_value):
    color_value = color_value.strip()
    rgb_match = re.match(r'rgb-?(\d+)-?(\d+)-?(\d+)', color_value, re.IGNORECASE)
    if rgb_match:
        r, g, b = map(int, rgb_match.groups())
        return '{:02X}{:02X}{:02X}'.format(r, g, b)
    try:
        return webcolors.name_to_hex(color_value.lower())
    except ValueError:
        pass
    if color_value.startswith('#') and len(color_value) == 7:
        return color_value.upper()
    return None

def convert_color_style_to_class(tag):
    style = tag.get('style', '')
    match = re.search(r'color\s*:\s*([^;]+)', style, re.IGNORECASE)
    if match:
        color_value = match.group(1).strip()
        if color_value.startswith('#'):
            class_color = f"sm-text-color-{color_value[1:].upper()}"
        else:
            sanitized = re.sub(r'[\s\(\)]', '', color_value).replace(',', '-')
            hex_color = color_to_hex(sanitized)
            class_color = f"sm-text-color-{hex_color}"
        new_style = re.sub(r'color\s*:\s*[^;]+;?', '', style, flags=re.IGNORECASE).strip()
        if new_style:
            tag['style'] = new_style
        else:
            del tag['style']
        if 'class' in tag.attrs:
            tag['class'].append(class_color)
        else:
            tag['class'] = [class_color]

def process_html_colors_preserve_html(html_text):
    soup = BeautifulSoup(html_text, "html.parser")
    for tag in soup.find_all(True):
        convert_color_style_to_class(tag)
    return str(soup)

def convert_element_to_storymap_object(el):
    img_tag = el.find('img')
    if img_tag:
        src = img_tag.get("src")
        if src and src.startswith("http://"):
            src = "https://" + src[len("http://"):]
        alt = img_tag.get("alt", "")
        link = ""
        figcaption = ""
        parent_figure = img_tag.find_parent("figure")
        if parent_figure:
            caption_tag = parent_figure.find("figcaption")
            if caption_tag:
                figcaption = caption_tag.get_text(strip=True)
        else:
            parent_div = img_tag.find_parent("div")
            if parent_div:
                caption_tag = parent_div.find("figcaption")
                if caption_tag:
                    figcaption = caption_tag.get_text(strip=True)
        img = Image(path=src)
        return img, figcaption, alt, link

    tag_name = el.name
    if tag_name == "p":
        inner_html = ''.join(str(c) for c in el.contents)
        processed_html = process_html_colors_preserve_html(inner_html)
        return Text(text=processed_html, style=TextStyles.PARAGRAPH)
    elif tag_name == "video":
        src = el.get("src")
        alt = el.get("alt", "")
        vid = Video(path=src)
        vid.alt_text = alt
        vid.caption = ""
        vid.video = src
        return vid
    elif tag_name == "audio":
        src = el.get("src")
        alt = el.get("alt", "")
        aud = Audio(path=src)
        aud.alt_text = alt
        aud.caption = ""
        aud.audio = src
        return aud
    elif tag_name in ["iframe", "embed"]:
        src = el.get("src") or el.get("data-src")
        alt = el.get("alt", "")
        if src:
            emb = Embed(path=src)
            emb.alt_text = alt
            emb.caption = ""
            emb.link = src
            return emb
    elif tag_name == "map":
        src = el.get("src")
        alt = el.get("alt", "")
        mp = Map(item="")
        mp.alt_text = alt
        mp.caption = ""
        mp.map = src
        return mp
    else:
        inner_html = ''.join(str(c) for c in el.contents)
        processed_html = process_html_colors_preserve_html(inner_html)
        return Text(text=processed_html, style=TextStyles.PARAGRAPH)

def parse_root_elements(html_snippet):
    soup = BeautifulSoup(html_snippet, "html.parser")
    html_elements = []
    for child in soup.contents:
        if not getattr(child, 'name', None):
            continue
        has_text = child.get_text(strip=True) != ""
        has_img = child.find('img') is not None
        has_video = child.find('video') is not None
        has_audio = child.find('audio') is not None
        has_iframe = child.find('iframe') is not None
        has_embed = child.find('embed') is not None
        has_map = child.find('map') is not None
        is_meaningful = has_text or has_img or has_video or has_audio or has_iframe or has_embed or has_map
        meaningful_children = []
        for c in child.children:
            if not getattr(c, 'name', None):
                continue
            c_has_text = c.get_text(strip=True) != ""
            c_has_img = c.find('img') is not None
            c_has_video = c.find('video') is not None
            c_has_audio = c.find('audio') is not None
            c_has_iframe = c.find('iframe') is not None
            c_has_embed = c.find('embed') is not None
            c_has_map = c.find('map') is not None
            if c_has_text or c_has_img or c_has_video or c_has_audio or c_has_iframe or c_has_embed or c_has_map:
                meaningful_children.append(c)
        if meaningful_children:
            html_elements.extend(meaningful_children)
            continue
        if is_meaningful:
            html_elements.append(child)
    return html_elements

def parse_nested_elements(html_snippet):
    soup = BeautifulSoup(html_snippet, "html.parser")
    soup_list = [child for child in soup.contents if getattr(child, 'name', None)]
    html_elements = []
    for element in soup_list:
        for c in element:
            if getattr(c, 'name', None):
                html_elements.append(c)
    return html_elements

def convert_html_elements_to_storymap_node(html_elements):
    content_nodes = []
    image_metadata = []
    for el in html_elements:
        node = convert_element_to_storymap_object(el)
        if isinstance(node, tuple):
            img, caption, alt, link = node
            content_nodes.append(img)
            image_metadata.append((img, caption, alt, link))
        elif node:
            content_nodes.append(node)
    return content_nodes, image_metadata

def download_thumbnail(webmap_item, thumbnail_path, gis=None):
    if webmap_item.thumbnail:
        url = f"{webmap_item._portal.resturl}content/items/{webmap_item.id}/info/{webmap_item.thumbnail}"
        token = gis._con.token if gis else None
        params = {'token': token} if token else {}
        response = requests.get(url, params=params)
        with open(thumbnail_path, 'wb') as f:
            f.write(response.content)
        return thumbnail_path
    else:
        print("No thumbnail available for this web map. Using default thumbnail.")
        return default_thumbnail_path

def create_image_thumbnail(image_url, thumbnail_path):
    response = requests.get(image_url)
    img = PILImage.open(BytesIO(response.content))
    img.thumbnail((800, 600))
    img.save(thumbnail_path)
    return thumbnail_path

def animate_progress_bar(progress_bar, stop_event):
    while not stop_event.is_set():
        for pct in range(0, 101):
            if stop_event.is_set():
                break
            progress_bar.value = pct
            time.sleep(0.02)
        for pct in range(100, -1, -1):
            if stop_event.is_set():
                break
            progress_bar.value = pct
            time.sleep(0.02)

## 3. Implement Core Conversion Logic as Functions

Encapsulate the main workflow into functions that accept parameters for user input.  
This enables both direct function calls (for debugging) and UI-driven execution.

In [ ]:
def connect_to_gis(agoNotebook=False, username=None):
    if agoNotebook:
        gis = GIS("home")
        print("Successfully logged in as:", gis.properties.user.username, "(role:", gis.properties.user.role, ")")
        return gis
    else:
        import keyring
        service_name = "system"
        success = False
        max_attempts = 5
        attempts = 0
        while not success and attempts < max_attempts:
            if username is None:
                username = input("Enter your ArcGIS Online username:")
            credential = keyring.get_credential(service_name, username)
            if credential is None:
                print(f"'{username}' is not in the local system's credential store. Try another username.")
                attempts += 1
                username = None
            else:
                password_from_keyring = keyring.get_password("system", username)
                portal_url = 'https://www.arcgis.com'
                gis = GIS(portal_url, username=username, password=password_from_keyring)
                success = True
                print("Successfully logged in as:", gis.properties.user.username, "(role:", gis.properties.user.role, ")")
        if not success:
            print("Maximum login attempts reached. Exiting.")
            sys.exit(1)
        return gis

def convert_classic_storymap(
    gis, 
    classic_storymap_id, 
    agoNotebook=False, 
    start_index=0, 
    end_index=None, 
    output_callback=None
):
    classic_item = Item(gis=gis, itemid=classic_storymap_id)
    classic_data = Item.get_data(classic_item)
    if classic_data == {}:
        raise Exception("ERROR: StoryMap to be converted must be hosted on ArcGIS Online.")
    elif type(classic_data) == dict:
        classic_item_json = json.dumps(classic_data)
        classic_item_data = json.loads(classic_item_json)
    else:
        classic_item_data = json.loads(classic_data)
    classic_story_settings = classic_item_data["values"]["settings"]
    classic_story_theme = classic_story_settings["theme"]
    classic_story_title = classic_item_data["values"]["title"]
    classic_story_data = classic_item_data["values"]["story"]
    entries = classic_story_data["entries"]
    classic_theme_group = classic_story_theme["colors"]["group"]
    if classic_theme_group == "dark":
        new_theme = Themes.OBSIDIAN
    elif classic_theme_group == "light":
        new_theme = Themes.SUMMIT
    else:
        new_theme = Themes.SUMMIT
    if end_index is None:
        end_index = len(entries) - 1
    created_storymaps = []
    published_storymap_items = []
    thumbnails = {}
    for i in range(start_index, end_index + 1):
        entry = entries[i]
        entry_title = entry.get("title")
        if output_callback:
            output_callback(f"Converting '{entry_title}' ...")
        else:
            print(f"Converting '{entry_title}' ...")
        story = StoryMap()
        story.theme(new_theme)
        sidecar = Sidecar(style="docked-panel")
        story.add(sidecar)
        media_info = entry.get("media", {})
        media_type = media_info.get("type")
        main_stage_content = None
        invalid_webmap = False
        if media_type == "webmap":
            webmap_id = media_info.get('webmap', {}).get('id')
            if webmap_id:
                try:
                    main_stage_content = Map(webmap_id)
                    invalid_webmap = False
                    thumbnails[i] = download_thumbnail(Item(gis=gis, itemid=webmap_id), f"webmap_thumbnail_{i}.png", gis)
                except Exception as e:
                    print(f"Error processing webmap {webmap_id}: {e}. WebMap may have been deleted or is otherwise inaccessible.")
                    invalid_webmap = True
        elif media_type == "webpage":
            webpage_url = media_info.get("webpage", {}).get("url")
            if webpage_url:
                main_stage_content = Embed(webpage_url)
        elif media_type == "image":
            image_url = media_info.get("image", {}).get("url")
            if image_url:
                main_stage_content = Image(image_url)
                thumbnails[i] = create_image_thumbnail(image_url=image_url, thumbnail_path=f"image_thumbnail_{i}.png")
        if i not in thumbnails:
            thumbnails[i] = Image(default_thumbnail_path)
        description_html = entry.get("description", "")
        content_nodes, content_image_metadata = convert_html_elements_to_storymap_node(parse_root_elements(description_html))
        text_panel = Text(content_nodes)
        sidecar.add_slide(contents=content_nodes, media=main_stage_content)
        for img, caption, alt, link in content_image_metadata:
            if caption:
                img.caption = caption
            if alt:
                img.alt_text = alt
            if link:
                img.link = link
        if not invalid_webmap and media_type == "webmap":
            extent_json = media_info.get('webmap', {}).get('extent')
            if extent_json:
                main_stage_content.set_viewpoint(extent=extent_json)
            old_layers = media_info.get('webmap', {}).get('layers', [])
            if old_layers and hasattr(main_stage_content, "map_layers"):
                for new_lyr in main_stage_content.map_layers:
                    for old_lyr in old_layers:
                        if new_lyr['id'] == old_lyr['id']:
                            new_lyr['visible'] = old_lyr['visibility']
        cover_properties = story.content_list[0]
        cover_properties.title = entry_title
        cover_properties.byline = ""
        cover_properties.date = "none"
        if i not in thumbnails or not thumbnails[i] or not os.path.isfile(getattr(thumbnails[i], "path", None)):
            thumbnails[i] = Image(default_thumbnail_path)
        try:
            cover_properties.media = Image(thumbnails[i].path)
        except Exception as e:
            print(f"Error creating Image object for {thumbnails[i]}: {e}")
        for k, v in story.properties['nodes'].items():
            if v['type'] == 'storycover':
                v['config'] = {'isHidden': 'true'}
        story_title = entry_title
        story.save(title=story_title, tags=["auto-created"], publish=True)
        created_storymaps.append(story)
        if hasattr(story, '_item'):
            published_story_item = story._item
            published_storymap_items.append(published_story_item)
        else:
            print("Could not find item for story:", story.title)
            continue
        story_url = "https://storymaps.arcgis.com/stories/" + published_story_item.id
        if output_callback:
            output_callback(f"Created '{published_story_item.title}'. URL: {story_url}")
        else:
            print(f"Created '{published_story_item.title}'. URL: {story_url}")
    return created_storymaps, published_storymap_items, thumbnails, new_theme, classic_item

def create_collection(
    gis, 
    classic_item, 
    published_storymap_items, 
    thumbnails, 
    new_theme, 
    agoNotebook=False, 
    output_callback=None
):
    classic_thumbnail = download_thumbnail(classic_item, "classic_story_thumbnail.png", gis)
    if not classic_thumbnail:
        classic_thumbnail = Image(default_thumbnail_path)
    collection = Collection()
    collection_title = classic_item.title
    for i, story in enumerate(published_storymap_items):
        collection.add(item=story, title=story.title, thumbnail=thumbnails[i])
    collection.content[0].title = collection_title
    collection.content[0].byline = ""
    collection.theme(new_theme)
    collection.content[1].type = "tab"
    collection.content[1].media = classic_thumbnail
    collection.save(title=collection_title, tags=["auto-created"], publish=True)
    published_collection_item = collection._item
    collection_url = "https://storymaps.arcgis.com/collections/" + published_collection_item.id
    if output_callback:
        output_callback(f"Created Collection '{collection_title}'. URL: {collection_url}")
    else:
        print(f"Created Collection '{collection_title}'. URL: {collection_url}")
    return collection

## 4. VS Code Debug/Test Entry Point

Set test parameters and call the core conversion functions directly.  
Use print statements for output, making it easy to debug in VS Code.

In [ ]:
# Only run this cell in VS Code for debugging!
if not agoNotebook:
    # Set test parameters here
    TEST_STORY_ID = "597d573e58514bdbbeb53ba2179d2359"  # Example: Katrina +10
    TEST_USERNAME = None  # Set to your AGO username or None to prompt
    TEST_START_INDEX = 0
    TEST_END_INDEX = 2  # Convert first 3 tabs for testing

    gis = connect_to_gis(agoNotebook=False, username=TEST_USERNAME)
    created_storymaps, published_storymap_items, thumbnails, new_theme, classic_item = convert_classic_storymap(
        gis, 
        classic_storymap_id=TEST_STORY_ID, 
        agoNotebook=False, 
        start_index=TEST_START_INDEX, 
        end_index=TEST_END_INDEX
    )
    # Optionally create collection
    create_collection(
        gis, 
        classic_item, 
        published_storymap_items, 
        thumbnails, 
        new_theme, 
        agoNotebook=False
    )

## 5. ipywidgets-based UI for ArcGIS Online Notebooks

Build an ipywidgets interface for user input and output.  
Outputs are displayed using widgets.Output().

In [ ]:
if widgets is not None:
    input_param1 = widgets.Checkbox(value=False, description="Are you running in ArcGIS Online Notebook?")
    input_param2 = widgets.Text(value="597d573e58514bdbbeb53ba2179d2359", description="Story ID:")
    start_index_widget = widgets.IntText(value=0, description="Start Tab Index:")
    end_index_widget = widgets.IntText(value=2, description="End Tab Index:")
    story_submit_btn = widgets.Button(description="Convert Story")
    collection_submit_btn = widgets.Button(description="Create Collection", disabled=True)
    story_output_box = widgets.Output()
    collection_output_box = widgets.Output()

    def on_story_submit(btn):
        story_output_box.clear_output()
        with story_output_box:
            try:
                agoNotebook_flag = input_param1.value
                story_id = input_param2.value
                start_idx = start_index_widget.value
                end_idx = end_index_widget.value
                gis = connect_to_gis(agoNotebook=agoNotebook_flag)
                def output_callback(msg):
                    print(msg)
                result = convert_classic_storymap(
                    gis, 
                    classic_storymap_id=story_id, 
                    agoNotebook=agoNotebook_flag, 
                    start_index=start_idx, 
                    end_index=end_idx,
                    output_callback=output_callback
                )
                global _created_storymaps, _published_storymap_items, _thumbnails, _new_theme, _classic_item
                _created_storymaps, _published_storymap_items, _thumbnails, _new_theme, _classic_item = result
                print(f"\nSuccess! Created {len(_created_storymaps)} StoryMaps.")
                collection_submit_btn.disabled = False
            except Exception as e:
                print(f"Error: {e}")

    def on_collection_submit(btn):
        collection_output_box.clear_output()
        with collection_output_box:
            try:
                agoNotebook_flag = input_param1.value
                gis = connect_to_gis(agoNotebook=agoNotebook_flag)
                def output_callback(msg):
                    print(msg)
                create_collection(
                    gis, 
                    _classic_item, 
                    _published_storymap_items, 
                    _thumbnails, 
                    _new_theme, 
                    agoNotebook=agoNotebook_flag,
                    output_callback=output_callback
                )
            except Exception as e:
                print(f"Error: {e}")

    story_submit_btn.on_click(on_story_submit)
    collection_submit_btn.on_click(on_collection_submit)

    display(widgets.VBox([
        input_param1, input_param2, start_index_widget, end_index_widget,
        story_submit_btn, story_output_box,
        collection_submit_btn, collection_output_box
    ]))

## 6. Conditional Execution: VS Code vs ArcGIS Online Notebook

Use the `agoNotebook` flag to switch between VS Code debug mode and ipywidgets UI mode.  
This enables seamless transition between development/debugging and deployment in ArcGIS Online Notebooks.

In [ ]:
# Set agoNotebook = True to enable ipywidgets UI (for ArcGIS Online Notebooks)
# Set agoNotebook = False to use VS Code debug/test entry point
# This flag can be set via the UI or manually in code.